In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Edge-IIoT/transfer_attack/attackmodel.csv", mode='a', index=False)




In [3]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/Edge-IIoT/transfer_attack/attackmodel.csv", mode='a', index=False, header=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_test = np.load('/home/jovyan/Edge-IIoT/x_test.npy')
x_train = np.load('/home/jovyan/Edge-IIoT/x_train.npy')
x_val = np.load('/home/jovyan/Edge-IIoT/x_val.npy')
y_test = np.load('/home/jovyan/Edge-IIoT/y_test.npy')
y_train = np.load('/home/jovyan/Edge-IIoT/y_train.npy')
y_val = np.load('/home/jovyan/Edge-IIoT/y_val.npy')

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [11]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        #self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(50, 30)
        #self.dropout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(30, 20)
        #self.dropout3 = nn.Dropout(0.1)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        #x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        #x = self.dropout3(x)
        x = self.fc4(x)
        return x

In [12]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Edge-IIoT/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [13]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [15]:
from art.attacks.evasion import ShadowAttack

In [17]:
x_test.shape[0]

4166

In [18]:
sigma_value =  [0.01, 0.1, 0.5, 1]

for sig in sigma_value:
    adv_crafter = ShadowAttack(classifier, sigma=sig, verbose=False, batch_size=1) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)
    filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_ShadowAttack_sig_{sig}.npy'
    np.save(filename, x_test_adv)

ValueError: This attack only accepts a single sample as input.

In [ ]:
sigma_value =  [0.01, 0.1, 0.5, 1]
for sig in sigma_value:
    filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_ShadowAttack_sig_{sig}.npy'
    x_test_adv = np.load(filename)
    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ShadowAttack', sig)

In [ ]:


# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [ ]:
# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# # Iterate over epsilon values
# for epsilon in epsilon_values:
#     adv_crafter = BasicIterativeMethod(classifier, eps=epsilon, max_iter = 10, verbose=False) # init eps=0.3
#     x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     np.save(filename, x_test_adv)

In [ ]:
# for epsilon in epsilon_values:
#     adv_crafter = FastGradientMethod(classifier, eps=epsilon) # init eps=0.3
#     x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
#     np.save(filename, x_test_adv)

In [ ]:
# for epsilon in epsilon_values:
#     adv_crafter = ProjectedGradientDescent(classifier, eps=epsilon, verbose=False) # init eps=0.3
#     x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
#     np.save(filename, x_test_adv)

In [ ]:
# DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
# for epsilon in DF_eps:
#     adv_crafter = DeepFool(classifier, epsilon=epsilon, verbose=False) # init eps=1e-06
#     x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
#     np.save(filename, x_test_adv)


In [ ]:
# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# # Iterate over epsilon values
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

# DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
# for epsilon in DF_eps:
#     filename = f'/home/jovyan/Edge-IIoT/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)